# TRABALHO 3

Modelação e implementação em SMT de um autómato híbrido que descreva o sistema de travagem ABS e verifique as suas propriedades dinâmicas.
O sistema usa 2 variáveis contínuas para descrever a  `velocidade do veículo` em relação ao solo e a `velocidade linear dos pneus` também em relação ao solo. O sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades. 
Como primeira abordagem a componente discreta do sistema contém os seguintes modos:
`Start`, `Free`, `Stopping`, `Blocked`, e `Stopped`.



## Autómatos híbridos


*Autómatos híbridos* são modelos de sistemas ciber-físicos. Essencialmente um autómato híbrido é uma *máquina de estados finita*, onde cada estado (designado por *modo de funcionamento*) descreve o comportamento contínuo de um sistema dinâmico modelado por *relações diferenciais ordinárias* (nas variáveis contínuas e nas suas derivadas em relação ao tempo) codificadas num predicado designado por *flow*. Estas variáveis contínuas evoluem num modo de funcionamento enquanto o seu *flow* for válido.
Cada transição discreta entre estados é representada por um arco anotado com um predicado (designado *switch* ou *jump*). Uma transição realiza-se quando o seu *switch* é válido. Ao ocorrer uma transição as variáveis internas dos modos conservam o seu valor, a não ser que lhes seja explicitamente atribuído um novo valor.

Um autómato híbrido pode ser descrito por um FOTS (sobre o qual podemos verificar propriedades lógicas com as metodologias que já estudamos). Nesse processo o FOTS vai *discretizar* as relações diferenciais ordinárias e vai mapea-las num espaço de estados também discreto.

### 1.  Defina um autómato híbrido que descreva a dinâmica do sistema de travagem ABS.

INSERIR IMAGEM E CONDIÇOES

### 2. Modele em lógica temporal LT  propriedades que, na sua opinião, caracterizam o comportamento desejável do sistema. Nomeadamente (mas não só”) a propriedade   ”o veículo para em menos de $t$ segundos” e “a velocidade $V$ diminui sempre com o tempo”.

INSERIR IMAGEM E CONDIÇOES

### 3. Codifique em SMT’s o modelo que definiu em 1.

In [1]:
from z3 import *

Comecemos por declarar os modos.
Na codificação em Z3 é conveniente usar um tipo enumerado para implementar os modos:

In [2]:
Mode,(Start, Free, Stopping, Blocked, Stopped) = EnumSort('Mode',('Start', 'Free', 'Stopping', 'Blocked', 'Stopped'))

Podemos agora declarar as variáveis do FOTS correspondente ao sistema ABS da seguinte forma:

In [3]:
def declare(i):
    s ={}
    s['M'] = Const('M'+str(i),Mode) # constante do tipo mode
    s['V'] = Real('V'+str(i))       # Velocidade do veículo
    s['v'] = Real('v'+str(i))       # velocidade da roda
    s['F'] = Real('F'+str(i))       # Força F
    s['f'] = Real('f'+str(i))       # força f
    s['P'] = Int('P'+str(i))        # peso
    s['a'] = Real('a'+str(i))       # força de atrito
    s['c'] = Real('c'+str(i))       # valor c
    #s['T'] = Real('T'+str(i))       # varíavel contínua que denota o tempo
    return s

De seguida iremos codificar os predicados Z3 `init`, `trans` e `inv`, que caracterizam, respectivamente, os estados iniciais, as transições e o invariante de modo do FOTS correspondente ao sistema ABS.

In [4]:
def init(s):
    return And(s['V'] == 20, s['v'] == 15, s['M'] == Start, s['F'] == s['c'] * (s['V'] - s['v']), s['a'] == 0.4, \
               s['P'] == 100, s['f'] == s['a'] * s['P'], s['c'] >= 0)

def inv(s):
    return s['V'] - s['v'] >= 0

In [5]:
def trans(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0, p['F'] == s['c'] * (s['V'] - s['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    constante_P = And(s['P'] == 100, p['P'] == s['P'])
    constante_a = And(s['a'] == 0.4, p['a'] == s['a'])
    
    # Equações com tempo (ao adicionar o tempo deixam de ser comuns e apenas ocorrem nos loops):
    #equacao_V = And(p['V'] - s['V'] == (s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']), p['T'] > s['T'])
    #equacao_v = And(p['v'] - s['v'] == (-s['a'] * s['P'] + s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']), \
    #                p['T'] > s['T'])
    
    # self-loops
    instart = And(s['M'] == Start, s['V'] == 20, s['v'] == 15, s['V'] - s['v'] > 0, s['c'] == 0, \
                  p['M'] == Start, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                 equacao_F, equacao_f, constante_P, constante_a)
    infree = And(s['M'] == Free, s['V'] - s['v'] > 0, s['c'] == 0, \
                 p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                equacao_F, equacao_f, constante_P, constante_a)
    instopping = And(s['M'] == Stopping, s['V'] - s['v'] > 0, s['c'] == 4, \
                     p['M'] == Stopping, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                    equacao_F, equacao_f, constante_P, constante_a)
    inblocked = And(s['M'] == Blocked, s['V'] > 0, s['v'] > 0, s['V'] - s['v'] == 0, s['c'] == 2, \
                    p['M'] == Blocked, p['v'] == s['v'], p['V'] == s['V'], p['c'] == s['c'], \
                   equacao_F, equacao_f, constante_P, constante_a)
    instopped = And(s['M'] == Stopped, s['V'] == 0, s['v'] == 0, s['c'] == 0, \
                    p['M'] == Stopped, p['V'] == s['V'], p['v'] == s['v'], \
                   equacao_F, equacao_f, constante_P, constante_a)
    
    # transições entre estados
    startTofree = And(s['M'] == Start, s['V'] >= 20, s['v'] >= 15, s['c'] == 0, \
                      p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                     equacao_F, equacao_f, constante_P, constante_a)
    freeTostopping = And(s['M'] == Free, s['V'] == 3 * s['v'], s['c'] == 0, \
                         p['M'] == Stopping, p['v'] == s['v'], p['c'] == 4, \
                        equacao_F, equacao_f, constante_P, constante_a)
    stoppingToblocked = And(s['M'] == Stopping, s['v'] == s['V'], s['c'] == 4, \
                            p['M'] == Blocked, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 2, \
                           equacao_F, equacao_f, constante_P, constante_a)
    blockedTofree = And(s['M'] == Blocked, s['V'] > s['v'], s['c'] == 2, \
                        p['M'] == Free, p['V'] < s['V'], p['v'] < s['v'], p['c'] == 0, \
                       equacao_F, equacao_f, constante_P, constante_a)
    stoppingTostopped = And(s['M'] == Stopping, s['V'] == s['v'], s['c'] == 2, \
                            p['M'] == Stopped, p['V'] == 0, p['v'] == 0, p['c'] == 0, \
                           equacao_F, equacao_f, constante_P, constante_a)
    
    return Or(instart, infree, instopping, inblocked, instopped, startTofree, freeTostopping, \
                  stoppingToblocked, blockedTofree, stoppingTostopped)

 - A transição "freeTostopping" nunca ocorre, uma vez que é preciso que V seja 3 vezes o valor de v mas nenhuma transição aumenta qualquer velocidade.

In [6]:
def trans_original(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0, p['F'] == s['c'] * (s['V'] * s['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    
    # self-loops
    instart = And(s['M'] == Start ,p['M'] == Start, s['V'] == 20, s['v'] == 15, p['v'] == s['v'], \
                  p['V'] == s['V'], s['V'] - s['v'] > 0, s['c'] == 0, p['c'] == s['c'], )
    infree = And(s['M'] == Free, p['M'] == Free, s['V'] - s['v'] > 0, p['v'] == s['v'], p['V'] == s['V'], \
                 s['c'] == 0, p['c'] == s['c'])
    instopping = And(s['M'] == Stopping, p['M'] == Stopping, p['v'] == s['v'], p['V'] == s['V'], s['c'] == 4, \
                     p['c'] == s['c'], s['V'] - s['v'] > 0 )
    inblocked = And(s['M'] == Blocked, p['M'] == Blocked, s['V'] - s['v'] == 0, s['v'] > 0, s['V'] > 0, \
                    p['v'] == s['v'], p['V'] == s['V'], s['c'] == 2, p['c'] == s['c'])
    instopped = And( s['M'] == Stopped, p['M'] == Stopped, p['v'] == s['v'], p['V'] == s['V'], s['v'] == 0, \
                    s['V'] == 0, s['c'] == 0)
    
    # transições entre estados
    startTofree = And(s['M'] == Start, p['M'] == Free, s['V'] >= 20, p['V'] == s['V'], s['v'] >= 15, \
                      p['v'] == s['v'], s['c'] == 0 , p['c'] == s['c'])
    freeTostopping = And(s['M'] == Free, p['M'] == Stopping, s['V'] == 3 * s['v'], p['v'] == s['v'], s['c'] == 0, \
                         p['c'] == 4)
    stoppingToblocked = And(s['M'] == Stopping, p['M'] == Blocked, s['v'] == s['V'], s['c'] == 4, p['c'] == 2, \
                            s['v'] == p['v'], s['V'] == p['V'])
    blockedTofree = And(s['M'] == Blocked, p['M'] == Free, s['V'] > s['v'], s['c'] == 2, p['c'] == 0)
    stoppingTostopped = And(s['M'] == Stopping, p['M'] == Stopped, s['V'] == s['v'], p['V'] == 0, p['v'] == 0, \
                            s['c'] == 2, p['c'] == 0)
    
    return And(Or(instart, infree, instopping, inblocked, instopped, startTofree, freeTostopping, \
                  stoppingToblocked, blockedTofree, stoppingTostopped), \
               And(equacao_F, equacao_f, p['V'] == s['c'] * (s['V'] - s['v']), \
                   p['v'] == - s['a'] * s['P'] + s['c'] * (s['V'] - s['v']), p['P'] == s['P'], p['a'] == s['a']))
    
    #return Or(And(And(And(And(And(And(instart, infree, instopping, inblocked, instopped, startTofree, \
    #                                  freeTostopping, stoppingToblocked, blockedTofree, stoppingTostopped, \
    #                                  p['V'] == s['c'] * (s['V'] - s['v'])), \
    #                              p['v'] == - s['a'] * s['P'] + s['c'] * (s['V'] - s['v'])), \
    #                          p['P'] == s['P'])), \
    #                  p['a'] == s['a']), \
    #              equacao_F), \
    #          equacao_f)
    

In [7]:
def gera_traco(declare,init,trans,inv,k):
    s = Solver()
    
    #criar k cópias do estado, guardar na lista do traço
    trace = []
    for i in range(k):
        trace.append(declare(i))
        
    #restriçoes init e trans    
    s.add(init(trace[0]))
    
    for i in range(k):
        s.add(inv(trace[i]))
    
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i + 1]))

    print(s.check())
    
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
                
gera_traco(declare, init, trans, inv, 50)

sat
0
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
1
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
2
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
3
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
4
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
5
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
6
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
7
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
8
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
9
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
10
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
11
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
12
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.0
13
M = Start
V = 20.0
v = 15.0
F = 0.0
f = 40.0
P = 100
a = 0.4
c = 0.